In [2]:
import os
import datetime
import pandas as pd
import re
import great_expectations
import agriculture as ag

In [2]:
#!pip install great_expectations

## Descargando los datos

Para descargar los datos referentes a los precios de diversos productos agricolas, podemos ejecutar directamente el script `agriculture.py` en la terminal de la siguiente manera:

```python3 agriculture.py```

El script mencionado llama de forma iterativa a la API de la pagina del Sistema Nacional de Informacion de Mercados (SNIIM) de la Secretaria de Economia, para recuperar los precios de diversos productos en diversos centros de distribucion en el pais, desde el año 2020 hasta la fecha.

Otra manera de descargar los datos es ejecutando la siguiente celda, la cual inicializa un objeto de tipo `ScrapperMarketAgriculture`, el cual contiene la funcion `scraping`, la cual realiza el mismo proceso del script mencionado anteriormente.

In [3]:
# ESTE SCRAPPER PUEDE TARDAR, RECOMENDABLE EJECUTAR DIRECTAMENTE DESDE TERMINAL
scrapper = ag.ScrapperMarketAgriculture() # Generamos un objeto de tipo ScrapperMarketAgriculture (scrapper SNIIM)
scrapper.scraping() # Scrapear la pagina del SNIIM para las frutas y hortalizas

Producto: Acelga - Primera
Producto: Aguacate Criollo - Primera
Producto: Aguacate Fuerte - Primera
Producto: Aguacate Hass - Calidad extra
Producto: Aguacate Hass - Calidad super extra
Producto: Aguacate Hass - Primera
Producto: Aguacate Hass adelantado - Primera
Producto: Aguacate Hass flor vieja - Primera
Producto: Aguacate Pagua - Primera
Producto: Ajo Blanco - Primera
Producto: Ajo Blanco # 8 - Primera
Producto: Ajo Morado - Primera
Producto: Apio - Primera
Producto: Berenjena - Primera
Producto: Betabel - Primera
Producto: Brócoli - Primera
Producto: Cacahuate - Primera
Producto: Calabacita Criolla - Primera
Producto: Calabacita Italiana - Primera
Producto: Calabacita regional - Primera
Producto: Calabaza de castilla - Primera
Producto: Camote - Primera
Producto: Caña - Primera
Producto: Cebolla Bola - Primera
Producto: Cebolla Bola grande - Primera
Producto: Cebolla de rabo - Primera
Producto: Cebolla Morada - Primera
Producto: Cilantro - Primera
Producto: Ciruela Amarilla - Pri

Una vez recuperados los datos, estos se almacenan de forma local en archivos .csv, los cuales se encuentran en el directorio `data/sniim`

In [3]:
# Directorio que contiene los archivos generados por el scrapper
sniim_dir = os.path.join("..","..","data","sniim")

## Procesamiento de los datos

Como al final tendremos una dataframe conteniendo los precios de diversos productos, podemos tratar de obtener el nombre de cada producto a partir del nombre del archivo csv, para esto definimos la siguiente funcion:

In [4]:
# Definimos una funcion para obtener el nombre del cultivo a partir del nombre del archivo csv
def estandarizar_nombre(nombre : str) -> str:
    """
    Regresa el nombre del cultivo/producto a partir del nombre del archivo .csv
    """
    cultivo_std = nombre.split('_')[:-1]
    cultivo_std = '_'.join(cultivo_std)
    cultivo_std = re.sub(r"(?<=#)_(?=\d)",'',cultivo_std)
    cultivo_std = re.sub("_"," ",cultivo_std)
    cultivo_std = re.sub("csemilla", "con semilla", cultivo_std)
    cultivo_std = re.sub("ssemilla", "sin semilla", cultivo_std)
    cultivo_std = cultivo_std.title()

    return cultivo_std

Finalmente, podemos iterar sobre los archivos presentes en el directorio anterior, para generar una dataframe por cada uno de ellos, agregar las dataframes a una lista para posteriormente concatenarlas en una sola.

Ademas, a cada dataframe le agregamos la columna `producto` para poder diferenciar las entradas una vez concatenadas las dataframes, y separamos la columna `destino` en dos: `destino` y `central` para identificar mas facilmente la Entidad de destino y la central de abastos donde se levanto la muestra.

In [5]:
# Almacenaremos nuestras dataframes en una lista para despues concatenar
df_list = []
for dirpath, subdir, files in os.walk(sniim_dir):
    for file in files:
        if file == ".DS_Store":
            continue
        
        df = pd.read_csv(os.path.join(dirpath,file)) 
        df["producto"] = estandarizar_nombre(file) # Obtenemos el nombre del producto a partir del nombre del archivo
        df[["destino","central"]] = df["destino"].str.split(": ", expand=True) # Separamos la columna destino en 2
        
        # Reordenamos las columnas
        cols = df.columns.to_list()
        del cols[-1]
        cols.insert(4,"central")
        
        del cols[-1]
        cols.insert(1,"producto")
        
        df = df[cols]
        
        # Agregamos la dataframe generada a la lista
        df_list.append(df)

In [6]:
precios_sniim = pd.concat(df_list,ignore_index=True) # Concatenamos nuestra lista de dataframes en una sola

Una vez obtenida nuestra dataframe final, podemos utilizar el metodo `sample` para obtener una vista general de nuestra dataframe.

In [7]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec,obs
794682,02/10/2023,Uva Superior,Caja de 9 kg.,Importación,Michoacán,Mercado de Abasto de Morelia,94.44,100.0,100.00,NaN
561964,07/04/2021,Cebolla Bola,Arpilla de 22 kg.,Sinaloa,Baja California,"Central de Abasto INDIA, Tijuana",5.45,5.45,5.45,NaN
476292,10/08/2023,Calabacita Criolla,Kilogramo,Sinaloa,Sinaloa,Central de Abasto de Culiacán,32.0,32.0,32.00,NaN
1626299,24/08/2022,Nopal,Ciento,Morelos,Morelos,Central de Abasto de Cuautla,0.7,0.8,0.80,NaN
1392865,27/03/2023,Mango Oro,Caja de 28 kg.,Veracruz,Veracruz,Central de Abasto de Jalapa,11.43,11.79,11.43,NaN
75495,13/11/2020,Chile Poblano,Kilogramo,Querétaro,Querétaro,Mercado de Abasto de Querétaro,21.0,23.0,22.00,NaN
1110987,05/10/2023,Tuna Blanca,Caja de 15 kg.,Puebla,Nuevo León,"Mercado de Abasto ""Estrella"" de San Nicolás de...",12.0,15.33,13.33,NaN
841659,30/06/2023,Chile Pimiento Morron,Kilogramo,Sonora,Sonora,"Mercado de Abasto ""Francisco I. Madero"" de Her...",55.0,60.0,60.00,NaN
382014,15/04/2020,Chayote Sin Espinas,Caja de 25 kg.,Puebla,Veracruz,Central de Abasto de Minatitlán,9.6,10.0,10.00,NaN
396503,30/07/2021,Camote,Caja de 15 kg.,Importación,Baja California,"Central de Abasto INDIA, Tijuana",20.0,20.0,20.00,NaN


Para una mayor facilidad al momento de comprender nuestra dataframe, generamos un diccionario de datos para esta, explicando lo que representa cada una de las columnas.

In [8]:
precios_dict = {
    "fecha": "Fecha en la que se levanto la encuesta",
    "product": "Producto al que hace referencia la entrada/muestra",
    "presentacion": "Presentacion del producto al que se hace referencia",
    "origen": "Entidad Federativa de donde proviene el producto",
    "destino": "Entidad Federativa hacia donde llego el producto",
    "central": "Central de abastos/centro de distribucion de donde se obtuvo la informacion",
    "precio_min": "El valor más bajo de la cotización dentro de una muestra (MXN)",
    "precio_max": "El valor más alto de la cotización dentro de una muestra (MXN)",
    "precio_frec": "Es el dato que más se repite en la muestra (moda) (MXN)",
    "obs": "Observaciones encontradas para la muestra"
}

In [16]:
dict_df = pd.DataFrame.from_dict(precios_dict,orient="index")
dict_df.columns = ["Descripcion"]
dict_df.rename_axis("Columna", inplace=True)
dict_df

,Descripcion
Columna,
fecha,Fecha en la que se levanto la encuesta
product,Producto al que hace referencia la entrada/mue...
presentacion,Presentacion del producto al que se hace refer...
origen,Entidad Federativa de donde proviene el producto
destino,Entidad Federativa hacia donde llego el producto
central,Central de abastos/centro de distribucion de d...
precio_min,El valor más bajo de la cotización dentro de u...
precio_max,El valor más alto de la cotización dentro de u...
precio_frec,Es el dato que más se repite en la muestra (mo...


In [19]:
dict_dir = os.path.join("..","..","diccionarios") 
if not os.path.exists(dict_dir):
    os.mkdir(dict_dir)
    
dict_df.to_csv(os.path.join(dict_dir,"diccionario_sniim.csv"),encoding="iso-8859-1")

## Limpieza de los datos

Una vez generado nuestro diccionario, podemos comenzar a tratar los datos de nuestro dataframe. Algo que podemos observar del sample de nuestro dataframe es que la columna `obs` parece ser la unica que cuenta con valores nulos. Podemos tener una idea de la proporcion de datos faltantes en esta columna ejecutando la siguiente celda, la cual calcula el porcentaje de entradas faltantes en la misma.

In [23]:
print(f"Porcentaje de entradas faltantes en columna 'obs': {100* precios_sniim['obs'].isna().sum() / precios_sniim.shape[0]:.2f}%")

KeyError: 'obs'

Vemos que la mayoria de las entradas en nuestra dataframe (> 90%) no cuenta con informacion en la columna `obs`, por lo que puede ser mas conveniente deshacernos de esta columna.

In [ ]:
precios_sniim.drop(columns="obs", inplace=True)

In [24]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
462007,14/12/2022,Cebolla Bola,Kilogramo,Morelos,San Luis Potosí,Centro de Abasto de San Luis Potosí,9.0,10.0,9.00
1540354,19/10/2022,Uva Globo,Caja de 8 kg.,Puebla,Veracruz,Central de Abasto de Minatitlán,50.0,55.0,52.50
1879338,13/01/2023,Piña Chica,Kilogramo,Nayarit,Nayarit,Nayarabastos de Tepic,16.0,18.0,17.00
400683,21/02/2022,Melon Cantaloupe #14,Caja de 14 kg.,Nuevo León,Tamaulipas,Módulo de Abasto de Reynosa,18.0,26.0,25.00
1654412,20/08/2020,Chile Ancho,Kilogramo,Zacatecas,Michoacán,Mercado de Abasto de Morelia,115.0,130.0,130.00
538430,13/10/2022,Fresa,Caja de 8 kg.,Michoacán,DF,Central de Abasto de Iztapalapa DF,35.0,45.0,40.00
554730,17/04/2020,Piña Mediana,Kilogramo,Chiapas,Coahuila,"Central de Abasto de La Laguna, Torreón",10.0,10.5,10.00
859666,24/11/2022,Platano Dominico,Kilogramo,Veracruz,Veracruz,Central de Abasto de Jalapa,18.0,19.0,18.00
1021800,11/08/2021,Platano Tabasco,Caja de 18 kg.,Chiapas,Chihuahua,Mercado de Abasto de Cd. Juárez,15.56,15.56,15.56
919231,17/01/2023,Papaya Maradol,Caja de 17 kg.,Chiapas,Chihuahua,Central de Abasto de Chihuahua,20.59,30.0,23.53


Posteriormente, podemos utilizar el metodo `info` de nuestra dataframe para obtener informacion acerca de los tipos de datos de las columnas o la posible presencia de mas datos faltantes.

In [25]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019574 entries, 0 to 2019573
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   fecha         object 
 1   producto      object 
 2   presentacion  object 
 3   origen        object 
 4   destino       object 
 5   central       object 
 6   precio_min    object 
 7   precio_max    object 
 8   precio_frec   float64
dtypes: float64(1), object(8)
memory usage: 138.7+ MB


Lo primero que notamos es que la columna `fecha` tiene un tipo de dato `object`, lo cual puede significar que esta almacenada como un string. Al tratarse de datos temporales, podemos convertir esta columna a un objeto de tipo `datetime` para una mayor facilidad al momento de manipular o explorar los datos.

In [26]:
precios_sniim["fecha"] = pd.to_datetime(precios_sniim["fecha"])

Despues podemos analizar los valores en la columna `origen` para verificar que no haya valores inesperados. Los posibles valores en esta columna son los nombres de los 32 estados de Mexico, o bien "Importacion" si el producto fue importado del extranjero, o "Nacional" si no hay datos sobre su estado de origen.

In [27]:
precios_sniim["origen"].unique()

array(['Nayarit', 'Distrito Federal', 'Oaxaca', 'Importación', 'Sonora',
       'Yucatán', 'Baja California', 'Campeche', 'San Luis Potosí',
       'Chiapas', 'Durango', 'Hidalgo', 'Michoacán', 'Jalisco',
       'Guanajuato', 'México', 'Quintana Roo', 'Sinaloa', 'Puebla',
       'Nuevo León', 'Aguascalientes', 'Zacatecas', 'Tamaulipas',
       'Chihuahua', 'Coahuila', 'Guerrero', 'Colima', 'Veracruz',
       'Querétaro', 'Tabasco', 'Morelos', 'Baja California Sur',
       'Nacional', 'Sin Especificar'], dtype=object)

Vemos que no existen anomalias aparentes en esta columna, asi que podemos proceder a revisar los valores de la columna `destino`, la cual deberia tener valores similares a la columna anterior.

In [28]:
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California ', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Nuevo León', 'Querétaro',
       'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora',
       'Tamaulipas', 'Veracruz', 'Aguascalientes', 'Zacatecas', 'Morelos',
       'Hidalgo', 'Puebla', 'Tabasco', 'Baja California Sur', 'Chihuahua',
       'Colima'], dtype=object)

Vemos que tampoco hay anomalias en esta columna. Sin embargo, algunos valores contienen espacios al final del valor (por ejemplo, `Baja California `). Esto se puede corregir facilmente utilizando la funcion `strip` que nos ofrece pandas:

In [29]:
precios_sniim["destino"] = precios_sniim["destino"].str.strip()
precios_sniim["destino"].unique()

array(['Nayarit', 'Oaxaca', 'Campeche', 'DF', 'México', 'Yucatán',
       'Baja California', 'Coahuila', 'Chiapas', 'Durango', 'Guanajuato',
       'Guerrero', 'Jalisco', 'Michoacán', 'Nuevo León', 'Querétaro',
       'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora',
       'Tamaulipas', 'Veracruz', 'Aguascalientes', 'Zacatecas', 'Morelos',
       'Hidalgo', 'Puebla', 'Tabasco', 'Baja California Sur', 'Chihuahua',
       'Colima'], dtype=object)

Finalmente, la columna `precio_max` aparece como de tipo `object`, por lo que posiblemente hay valores que impidieron que fuera parseada a `float`.

Despues de una inspeccion de esta columna, vemos que para algunos precios, se utiliza la coma (,) para separar los millares. Podemos eliminar esta coma de nuestro dataframe con el metodo `replace`, para despues transformarla a tipo `float` con el metodo `astype(float)`.

In [30]:
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(str).str.replace(",","")
precios_sniim["precio_max"] = precios_sniim["precio_max"].astype(float)

In [31]:
precios_sniim["precio_min"] = precios_sniim["precio_min"].astype(str).str.replace(",","")
precios_sniim["precio_min"] = precios_sniim["precio_min"].astype(float)

Finalmente, podemos volver a obtener una muestra de nuestro dataframe para ver su estructura final, asi como volver a llamar al metodo `info` para verificar los tipos de datos de cada columna y que no haya valores faltantes.

In [32]:
precios_sniim.sample(10)

,fecha,producto,presentacion,origen,destino,central,precio_min,precio_max,precio_frec
402175,2022-03-11,Ajo Blanco,Kilogramo,Baja California,Baja California Sur,Unión de Comerciantes de La Paz,85.00,150.00,150.00
1538045,2022-12-14,Uva Globo,Kilogramo,Nayarit,Nayarit,Nayarabastos de Tepic,68.00,70.00,69.00
158345,2023-03-22,Chile Serrano,Kilogramo,Sinaloa,San Luis Potosí,Centro de Abasto de San Luis Potosí,23.00,25.00,23.00
1314528,2021-08-20,Tomate Saladette,Kilogramo,Jalisco,Colima,Centros de distribución de Colima,16.00,22.00,22.00
127712,2022-10-13,Chile Hungaro,Kilogramo,Nayarit,Nayarit,Mercado de abasto 'Adolfo López Mateos' de Tepic,35.00,37.00,36.00
1462460,2022-04-28,Tomate Saladette,Caja de 32 kg.,Sinaloa,Aguascalientes,Centro Comercial Agropecuario de Aguascalientes,8.13,9.38,8.75
1840781,2020-10-14,Chile De Arbol Seco,Kilogramo,Importación,Veracruz,Central de Abasto de Jalapa,90.00,95.00,95.00
1457684,2022-01-28,Naranja Agria,Caja de 20 kg.,Yucatán,Yucatán,"Mercado ""Casa del Pueblo""",3.50,3.50,3.50
1574923,2021-06-22,Toronja Roja,Kilogramo,Michoacán,Morelos,Central de Abasto de Cuautla,16.00,18.00,17.00
184260,2022-03-24,Chile Poblano,Kilogramo,Quintana Roo,Quintana Roo,"Mercado de Chetumal, Quintana Roo",38.00,38.00,38.00


In [33]:
precios_sniim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2019574 entries, 0 to 2019573
Data columns (total 9 columns):
 #   Column        Dtype         
---  ------        -----         
 0   fecha         datetime64[ns]
 1   producto      object        
 2   presentacion  object        
 3   origen        object        
 4   destino       object        
 5   central       object        
 6   precio_min    float64       
 7   precio_max    float64       
 8   precio_frec   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 138.7+ MB


Finalmente, almacenamos nuestro dataframe tidy en un archivo `.parquet`.

In [34]:
clean_data_dir = os.path.join("..","..","data","clean_data")

if not os.path.exists(clean_data_dir):
    os.mkdir(clean_data_dir)
    
precios_sniim.to_parquet(os.path.join(clean_data_dir,"precios_sniim.parquet"))